# 例4.1「格子世界」（図4.2）

反復方策評価の実装を行う。

In [1]:
import numpy as np
import copy

class GridWorld:
    """格子世界を定義するクラス"""
    
    # 定数定義
    ACTIONS = ["up", "down", "right", "left"] # 可能なアクションの定義
    REWARD = -1.0  # 期待報酬関数（任意のs,s',aに対して-1とする）
    PRINTABLE_ITERATIONS = [0, 1, 2, 3, 10] # 出力する回
    THETA = 0.0001 # 停止判定基準の値（正の小さな値）
    
    def __init__(self):
        """コンストラクタ"""
        
        # ラベルごと、アクションごとの状態遷移先を定義
        self.transitions = {
             1: { "up": 1 , "down": 5 , "right": 2 , "left": 0 },
             2: { "up": 2 , "down": 6 , "right": 3 , "left": 1 },
             3: { "up": 3 , "down": 7 , "right": 3 , "left": 2 },
             4: { "up": 0 , "down": 8 , "right": 5 , "left": 4 },
             5: { "up": 1 , "down": 9 , "right": 6 , "left": 4 },
             6: { "up": 2 , "down": 10, "right": 7 , "left": 5 },
             7: { "up": 3 , "down": 11, "right": 7 , "left": 6 },
             8: { "up": 4 , "down": 12, "right": 9 , "left": 8 },
             9: { "up": 5 , "down": 13, "right": 10, "left": 8 },
            10: { "up": 6 , "down": 14, "right": 11, "left": 9 },
            11: { "up": 7 , "down": 0 , "right": 11, "left": 10 },
            12: { "up": 8 , "down": 12, "right": 13, "left": 12 },
            13: { "up": 9 , "down": 13, "right": 14, "left": 12 },
            14: { "up": 10, "down": 14, "right": 0 , "left": 13 },
        }
    
    def expected_reward(self, s_from, s_to, action):
        """期待報酬関数"""
        return self.REWARD # すべて「-1」

    def initialize_v(self):
        """状態価値ベクトルを初期化する"""
        self.values = np.array([0.0 for i in range(16)])

    def print_v(self, values, shaped=True):
        """状態価値ベクトルを出力する"""
        if shaped:
            print(np.round(np.reshape(values, (4, 4)), 1))
        else:
            print(np.round(values), 1)

    def policy_evaluation(self, values, sweep=False):
        """方策評価のアルゴリズム実装"""
        old_values = copy.copy(values) # 元の価値
        new_values = copy.copy(values) # 新しい価値
        for i in self.transitions.keys(): # 終端(0と15)は更新しない
            reward_sum = 0.0
            for a in self.ACTIONS:
                # アクションごとに、報酬値として-1に遷移先の状態価値を足し込む
                reward_sum += self.expected_reward(i, self.transitions[i][a], a) + old_values[self.transitions[i][a]]
            # 状態価値を更新
            new_values[i] = 0.25 * reward_sum # 特定のアクションをとる確率0.25を掛け、報酬の期待値を得る
            if sweep:
                old_values[i] = new_values[i] # その場更新する場合
        return new_values

    def run_iterative_policy_evaluation(self, sweep=False, max=None, shaped=True):
        """反復方策評価のアルゴリズム実装"""
        # 評価対象の方策πを入力
        # 状態価値ベクトルを初期化する
        self.initialize_v()
        if 0 in self.PRINTABLE_ITERATIONS:
            print("## 初期値:")
            self.print_v(self.values, shaped) # 初期値を出力
        k = 1
        delta = 0.0
        while True:
            old_values = copy.copy(self.values)
            self.values = self.policy_evaluation(old_values, sweep)
            if k in self.PRINTABLE_ITERATIONS:
                print("## %d回目:" % k)
                self.print_v(self.values, shaped) # k回目の更新値を出力
            k += 1
            if max is None:
                # maxが指定されていない場合は、deltaが停止判定基準になるまで実行する
                delta = np.max(np.absolute(self.values - old_values))
                if delta < self.THETA:
                    print("## %d回目（収束）:" % k)
                    self.print_v(self.values, shaped) # 収束回の更新値を出力
                    break
            if max is not None and k > max:
                # maxが指定されている場合は、max回まで実行する
                break
        return self.values

In [2]:
# 実行プログラム
gw = GridWorld()

print('# スイープ操作（その場更新）しない場合:')
gw.run_iterative_policy_evaluation()

# スイープ操作（その場更新）しない場合:
## 初期値:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
## 1回目:
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
## 2回目:
[[ 0.  -1.8 -2.  -2. ]
 [-1.8 -2.  -2.  -2. ]
 [-2.  -2.  -2.  -1.8]
 [-2.  -2.  -1.8  0. ]]
## 3回目:
[[ 0.  -2.4 -2.9 -3. ]
 [-2.4 -2.9 -3.  -2.9]
 [-2.9 -3.  -2.9 -2.4]
 [-3.  -2.9 -2.4  0. ]]
## 10回目:
[[ 0.  -6.1 -8.4 -9. ]
 [-6.1 -7.7 -8.4 -8.4]
 [-8.4 -8.4 -7.7 -6.1]
 [-9.  -8.4 -6.1  0. ]]
## 174回目（収束）:
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]


array([  0.        , -13.99893866, -19.99842728, -21.99824003,
       -13.99893866, -17.99861452, -19.9984378 , -19.99842728,
       -19.99842728, -19.9984378 , -17.99861452, -13.99893866,
       -21.99824003, -19.99842728, -13.99893866,   0.        ])

In [3]:
print('# スイープ操作（その場更新）する場合:')
gw.run_iterative_policy_evaluation(sweep=True)

# スイープ操作（その場更新）する場合:
## 初期値:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
## 1回目:
[[ 0.  -1.  -1.2 -1.3]
 [-1.  -1.5 -1.7 -1.8]
 [-1.2 -1.7 -1.8 -1.9]
 [-1.3 -1.8 -1.9  0. ]]
## 2回目:
[[ 0.  -1.9 -2.5 -2.7]
 [-1.9 -2.8 -3.2 -3.4]
 [-2.5 -3.2 -3.6 -3.2]
 [-2.7 -3.4 -3.2  0. ]]
## 3回目:
[[ 0.  -2.8 -3.8 -4.2]
 [-2.8 -4.  -4.7 -4.9]
 [-3.8 -4.7 -5.  -4.3]
 [-4.2 -4.9 -4.3  0. ]]
## 10回目:
[[  0.   -7.8 -11.1 -12.2]
 [ -7.8 -10.4 -11.8 -11.9]
 [-11.1 -11.8 -11.1  -8.8]
 [-12.2 -11.9  -8.8   0. ]]
## 115回目（収束）:
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]


array([  0.        , -13.99931242, -19.99901152, -21.99891199,
       -13.99931242, -17.99915625, -19.99908389, -19.99909436,
       -19.99901152, -19.99908389, -17.99922697, -13.99942284,
       -21.99891199, -19.99909436, -13.99942284,   0.        ])

## 練習問題4.1

$Q^\pi(s,a)=\sum_{s'}^{}{R^a_{s,s'}P^a_{s,s'}}+\gamma\sum_{s'}^{}{P^a_{s,s'}V^\pi(s')}$ 

under policy $\pi$, any $R=-1, \gamma=0.9$

$Q^{\pi}(11,down) = R^{down}_{11,0}P^{down}_{11,0}+{\gamma}P^{down}_{11,0}V^\pi(0) = -1\times1+0.9\times1\times0 = -1$

$Q^{\pi}(7,down) = R^{down}_{7,11}P^{down}_{7,11}+{\gamma}P^{down}_{7,11}V^\pi(11) = -1\times1+0.9\times1\times-14 = -13.6$

## 練習問題4.2

状態13の真下に新しい状態15を追加し、状態15で行動→状態が down→15, left→12, right→14, up→13 へ遷移するものとした場合の $V^\pi(15)$ を計算する。

In [4]:
gw2 = GridWorld()
gw2.transitions[15] = {'down': 15, 'left': 12, 'right': 14, 'up': 13}
gw2.transitions

{1: {'down': 5, 'left': 0, 'right': 2, 'up': 1},
 2: {'down': 6, 'left': 1, 'right': 3, 'up': 2},
 3: {'down': 7, 'left': 2, 'right': 3, 'up': 3},
 4: {'down': 8, 'left': 4, 'right': 5, 'up': 0},
 5: {'down': 9, 'left': 4, 'right': 6, 'up': 1},
 6: {'down': 10, 'left': 5, 'right': 7, 'up': 2},
 7: {'down': 11, 'left': 6, 'right': 7, 'up': 3},
 8: {'down': 12, 'left': 8, 'right': 9, 'up': 4},
 9: {'down': 13, 'left': 8, 'right': 10, 'up': 5},
 10: {'down': 14, 'left': 9, 'right': 11, 'up': 6},
 11: {'down': 0, 'left': 10, 'right': 11, 'up': 7},
 12: {'down': 12, 'left': 12, 'right': 13, 'up': 8},
 13: {'down': 13, 'left': 12, 'right': 14, 'up': 9},
 14: {'down': 14, 'left': 13, 'right': 0, 'up': 10},
 15: {'down': 15, 'left': 12, 'right': 14, 'up': 13}}

In [5]:
gw2.run_iterative_policy_evaluation(sweep=True)

## 初期値:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
## 1回目:
[[ 0.  -1.  -1.2 -1.3]
 [-1.  -1.5 -1.7 -1.8]
 [-1.2 -1.7 -1.8 -1.9]
 [-1.3 -1.8 -1.9 -2.2]]
## 2回目:
[[ 0.  -1.9 -2.5 -2.7]
 [-1.9 -2.8 -3.2 -3.4]
 [-2.5 -3.2 -3.6 -3.2]
 [-2.7 -3.4 -3.2 -3.9]]
## 3回目:
[[ 0.  -2.8 -3.8 -4.2]
 [-2.8 -4.  -4.7 -4.9]
 [-3.8 -4.7 -5.  -4.3]
 [-4.2 -4.9 -4.3 -5.3]]
## 10回目:
[[  0.   -7.8 -11.1 -12.2]
 [ -7.8 -10.4 -11.8 -11.9]
 [-11.1 -11.8 -11.1  -8.8]
 [-12.2 -11.9  -8.8 -12.1]]
## 115回目（収束）:
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14. -20.]]


array([  0.        , -13.99931242, -19.99901152, -21.99891199,
       -13.99931242, -17.99915625, -19.99908389, -19.99909436,
       -19.99901152, -19.99908389, -17.99922697, -13.99942284,
       -21.99891199, -19.99909436, -13.99942284, -19.99911612])

以上により、 $V^\pi(15)=-20$ となる。

さらに、状態13での行動downが、新しい状態15に遷移するようにダイナミクスが変化した場合の $V^\pi(15)$ を計算する。

In [6]:
gw3 = GridWorld()
gw3.transitions = copy.copy(gw2.transitions)
gw3.transitions[13]['down'] = 15
gw3.transitions

{1: {'down': 5, 'left': 0, 'right': 2, 'up': 1},
 2: {'down': 6, 'left': 1, 'right': 3, 'up': 2},
 3: {'down': 7, 'left': 2, 'right': 3, 'up': 3},
 4: {'down': 8, 'left': 4, 'right': 5, 'up': 0},
 5: {'down': 9, 'left': 4, 'right': 6, 'up': 1},
 6: {'down': 10, 'left': 5, 'right': 7, 'up': 2},
 7: {'down': 11, 'left': 6, 'right': 7, 'up': 3},
 8: {'down': 12, 'left': 8, 'right': 9, 'up': 4},
 9: {'down': 13, 'left': 8, 'right': 10, 'up': 5},
 10: {'down': 14, 'left': 9, 'right': 11, 'up': 6},
 11: {'down': 0, 'left': 10, 'right': 11, 'up': 7},
 12: {'down': 12, 'left': 12, 'right': 13, 'up': 8},
 13: {'down': 15, 'left': 12, 'right': 14, 'up': 9},
 14: {'down': 14, 'left': 13, 'right': 0, 'up': 10},
 15: {'down': 15, 'left': 12, 'right': 14, 'up': 13}}

In [7]:
gw3.run_iterative_policy_evaluation(sweep=True)

## 初期値:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
## 1回目:
[[ 0.  -1.  -1.2 -1.3]
 [-1.  -1.5 -1.7 -1.8]
 [-1.2 -1.7 -1.8 -1.9]
 [-1.3 -1.8 -1.9 -2.2]]
## 2回目:
[[ 0.  -1.9 -2.5 -2.7]
 [-1.9 -2.8 -3.2 -3.4]
 [-2.5 -3.2 -3.6 -3.2]
 [-2.7 -3.5 -3.2 -3.9]]
## 3回目:
[[ 0.  -2.8 -3.8 -4.2]
 [-2.8 -4.  -4.7 -4.9]
 [-3.8 -4.7 -5.  -4.3]
 [-4.2 -5.  -4.3 -5.4]]
## 10回目:
[[  0.   -7.8 -11.1 -12.2]
 [ -7.9 -10.5 -11.8 -11.9]
 [-11.2 -11.8 -11.1  -8.8]
 [-12.3 -12.   -8.9 -12.2]]
## 115回目（収束）:
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14. -20.]]


array([  0.        , -13.9993636 , -19.99908399, -21.99899095,
       -13.99936685, -17.99922185, -19.99915318, -19.9991615 ,
       -19.99909265, -19.99915945, -17.99928843, -13.99946691,
       -21.99900476, -19.99917652, -13.99947208, -19.99919296])

以上により、やはり $V^\pi(15)=-20$ となる。